In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('output.csv')

In [3]:
df.head()

,tweet_id,tweet_text,label
0,'504075795435094016',"Bridges, overpasses earn approval after Northe...",infrastructure_and_utilities_damage
1,'503765744333901824',"Say what you want, but the earthquake that hit...",other_useful_information
2,'503741064067313665',Cool data analysis of today's Napa earthquake ...,other_useful_information
3,'503910039715725312',California assessing damage after large quake ...,infrastructure_and_utilities_damage
4,'503892224589848576',RT @heyyouapp: California USA Rancho Cucamonga...,caution_and_advice


In [4]:
df.shape

(1701, 3)

In [5]:
df['label'].value_counts()

,count
label,
other_useful_information,868
infrastructure_and_utilities_damage,305
injured_or_dead_people,183
not_related_or_irrelevant,106
sympathy_and_emotional_support,80
donation_needs_or_offers_or_volunteering_services,77
caution_and_advice,72
missing_trapped_or_found_people,6
displaced_people_and_evacuations,4


In [6]:
# converting to lists
X= list(df['tweet_text'])
Y= list(df['label'])

In [7]:
# label conding of Target Variable:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)

In [8]:
print(Y)

[3 7 7 ... 4 3 7]


In [9]:
# Train-test Splitting:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
#Installing Transformers:

!pip install transformers

In [11]:
# 1> call the pretrained model
# 2> Call the related tokenizer
# loading the pretrained DistilBERT model for sequence classification.
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [13]:
# Convert Tokenized Data into TensorFlow Dataset Objects:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)

In [16]:
from transformers import TFDistilBertForSequenceClassification,DistilBertTokenizer

# Determine the number of classes in your dataset
num_labels = len(set(y_train))

# Define your distribution strategy
strategy = tf.distribute.MirroredStrategy()

# Use the strategy's scope to create and compile the model
with strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_labels)
    # Compile the model with an optimizer, loss function, and metrics
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [17]:
# Train the model
history= model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3
85/85 [==============================] - 724s 8s/step - loss: 1.3710 - accuracy: 0.5801 - val_loss: 0.9115 - val_accuracy: 0.7243
Epoch 2/3
85/85 [==============================] - 685s 8s/step - loss: 0.7545 - accuracy: 0.7750 - val_loss: 0.7444 - val_accuracy: 0.7566
Epoch 3/3
85/85 [==============================] - 674s 8s/step - loss: 0.5412 - accuracy: 0.8279 - val_loss: 0.7660 - val_accuracy: 0.7566


In [ ]:
# You can monitor the training progress using the history object.
# It contains the training and validation loss and accuracy for each epoch.
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [19]:
#Evaluation: Once the training is complete, evaluate your model's performance on a held-out test set (if you have one separate from your validation set).
#  Use the model.evaluate() method to calculate metrics like accuracy, precision, recall, or F1-score.
#  Evaluate the model on a separate test set (if available)
loss, accuracy = model.evaluate(test_dataset)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

22/22 [==============================] - 76s 3s/step - loss: 0.7660 - accuracy: 0.7566
Test loss: 0.7660411596298218
Test accuracy: 0.7565982341766357


In [ ]:
# prompt: evaluate f1 score and confusion matrix

from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns

# Get predictions
y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate F1-score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
print("F1-score:", f1)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [21]:
# Make predictions on new data
new_text = ["RT @nbcbayarea: Napa: Skaters find upside to quake damage. Video: http://t.co/vsKpdwYZWK (Pic: Jeremy Carroll) http://t.co/wcEqkR1RtW"]
new_encodings = tokenizer(new_text, truncation=True, padding=True)
new_dataset = tf.data.Dataset.from_tensor_slices(dict(new_encodings)).batch(1)
predictions = model.predict(new_dataset)

# Process predictions to get the predicted class labels (e.g., using argmax)
predicted_labels = np.argmax(predictions.logits, axis=1)
print(predicted_labels)

1/1 [==============================] - 3s 3s/step
[7]


In [22]:
# Convert label indices to original labels
predicted_labels_text = le.inverse_transform(predicted_labels)
print("Predicted labels (text):", predicted_labels_text)


Predicted labels (text): ['other_useful_information']


In [ ]:
# Fine-tuning and Optimization (Optional): If you are not satisfied with the model's performance, you can further fine-tune it:

# Hyperparameter Tuning: Experiment with different hyperparameters like learning rate, batch size, and number of epochs to find the optimal settings.
# Layer Freezing: Freeze certain layers of the pre-trained model and train only the remaining layers to prevent overfitting on smaller datasets.
# Data Augmentation: Increase the size and diversity of your training data by applying techniques like text augmentation.
# Deployment (Optional): If you want to use your model in a real-world application, you'll need to deploy it. This could involve creating an API or integrating it into a larger system.

# Remember to save your trained model using model.save() so you can reuse it later without retraining.

In [23]:
# Save the model to a directory
model.save_pretrained('saved_model')
tokenizer.save_pretrained('saved_model')


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

Integration:

In [38]:
# pip install Flask transformers tensorflow


In [37]:
from flask import Flask, request, jsonify
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
import tensorflow as tf
import numpy as np

app = Flask(__name__)

# Load the pretrained BERT model and tokenizer
model = TFDistilBertForSequenceClassification.from_pretrained('saved_model')
tokenizer = DistilBertTokenizer.from_pretrained('saved_model')

# Define a route for predictions
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get JSON data from request
        data = request.json
        text = data.get('text', '')

        # Tokenize the text
        encodings = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors='tf')
        input_ids = encodings['input_ids']

        # Make prediction
        predictions = model(input_ids)
        logits = predictions.logits
        predicted_label = np.argmax(logits, axis=1).tolist()

        return jsonify({'predicted_label': predicted_label})

    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True)


Some layers from the model checkpoint at saved_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at saved_model and are newly initialized: ['dropout_199']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# prompt: summarise the entire steps we have followed from 1st line of code and what all errors we faced and how we have tackled all the errors.

# Summary of Steps and Error Handling

# 1. Data Loading and Preprocessing:
#    - Loaded data from 'output.csv' into a pandas DataFrame.
#    - Explored data using df.head() and df.shape.
#    - Checked class distribution with df['label'].value_counts().
#    - Converted text and labels to lists for further processing.
#    - Used LabelEncoder to encode categorical labels.

# 2. Train-Test Split:
#    - Split data into training and testing sets using train_test_split.

# 3. Model Selection and Tokenization:
#    - Installed transformers library for using DistilBERT model.
#    - Loaded DistilBertTokenizerFast for tokenizing text.
#    - Tokenized training and testing data.

# 4. Dataset Creation:
#    - Converted tokenized data into TensorFlow Dataset objects for efficient training.

# 5. Model Definition and Compilation:
#    - Determined the number of classes for classification.
#    - Used MirroredStrategy for distributed training.
#    - Loaded TFDistilBertForSequenceClassification model.
#    - Compiled the model with Adam optimizer and appropriate loss and metrics.

# 6. Model Training:
#    - Trained the model on the training dataset, validating on the test dataset for a specified number of epochs.
#    - Monitored training progress using the history object.

# 7. Model Evaluation:
#    - Plotted training and validation accuracy and loss curves.
#    - Evaluated the model on the test dataset using model.evaluate().
#    - Calculated F1-score and confusion matrix for performance assessment.
#    - Visualized confusion matrix using seaborn heatmap.

# 8. Prediction on New Data:
#    - Tokenized new text data.
#    - Made predictions using the trained model.
#    - Converted predicted label indices to original labels.

# 9. Model Saving:
#    - Saved the trained model and tokenizer for later use.

# 10. Flask API Development:
#    - Installed Flask and other necessary libraries.
#    - Created a Flask app to serve predictions.
#    - Loaded the saved model and tokenizer.
#    - Defined a route for handling prediction requests.
#    - Implemented error handling for robust API functionality.

# Errors Encountered and Solutions:

# - No major errors were explicitly mentioned in the provided code.
# - Potential errors and solutions in a typical workflow:
#   - Missing libraries: Install required libraries using pip.
#   - Shape mismatches: Ensure data dimensions are compatible for model input.
#   - Out of memory errors: Adjust batch size or use a more powerful GPU.
#   - Overfitting: Apply regularization, data augmentation, or early stopping.
#   - Deployment issues: Debug and resolve errors related to server configuration, dependencies, or network connectivity.
